In [31]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import keras

import matplotlib.pyplot as plt
import openpyxl
import cpp_complexity as cppcom
import data

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K





In [32]:

folderList = [chr(i) for i in range(65,77)]
folderList = ['01','02','03','04','05','06','07','08','09','10']
def read_excel(filename):
    list = []
    for i in range(len(folderList)):
        list.append(pd.read_excel(filename,sheet_name=i,engine='openpyxl'))
    return list


def make_distribution(list, num, col):
    plot_max = min(list[num][col])
    x = list[num][col] == plot_max
    print(list[num][x])
    plot_max = max(list[num][col])
    x = list[num][col] == plot_max
    print(list[num][x])

    plotList = [0 for _ in range(plot_max+5)]
    for i in list[num][col]:
        plotList[i] += 1
    plt.plot(plotList)
    plt.show()




In [43]:
df = read_excel('ml_labeled_data.xlsx')

In [44]:
def make_standard_df(df,input_col_list):
    for i in range(len(df)):
        # E문제에 valid한 코드가 1개 밖에 없으므로 normalize식 nan 발생, 따라서 미리 제거
        if folderList[i] == 'E':
            continue
        if i == 0:
            normal_df = (df[i][input_col_list] - df[i][input_col_list].mean())/df[i][input_col_list].std()
            normal_df['filename'] = df[i]['filename']
            normal_df['label'] = df[i]['label']

        else:
            temp_df = (df[i][input_col_list] - df[i][input_col_list].mean())/df[i][input_col_list].std()
            temp_df['filename'] = df[i]['filename']
            temp_df['label'] = df[i]['label']
            normal_df = pd.concat([normal_df,temp_df],ignore_index=True)
    return normal_df

In [45]:
def make_minmax_df(df,input_col_list):
    for i in range(len(df)):
        # E문제에 valid한 코드가 1개 밖에 없으므로 normalize식 nan 발생, 따라서 미리 제거
        if folderList[i] == 'E':
            continue
        if i == 0:
            normal_df = (df[i][input_col_list] - df[i][input_col_list].min())/(df[i][input_col_list].max()-df[i][input_col_list].min())
            normal_df['filename'] = df[i]['filename']
            normal_df['label'] = df[i]['label']

        else:
            temp_df = (df[i][input_col_list] - df[i][input_col_list].min())/(df[i][input_col_list].max()-df[i][input_col_list].min())
            temp_df['filename'] = df[i]['filename']
            temp_df['label'] = df[i]['label']
            normal_df = pd.concat([normal_df,temp_df],ignore_index=True)
    return normal_df

In [46]:
input_col =['distinct_func', 'number_func','distinct_var','number_var','depth','loc']
normal_df = make_standard_df(df,input_col)

In [47]:
print(normal_df)

     distinct_func  number_func  distinct_var  number_var     depth       loc  \
0        -0.609636    -0.597224     -0.294281   -0.522622  1.203297 -0.089062   
1        -0.321943    -0.002361     -0.139784   -0.193041  1.203297  0.404206   
2        -0.034249    -0.002361      0.014714   -0.291915 -1.355613 -0.089062   
3         1.404218     1.782230      2.795671    5.805347 -1.355613  4.103723   
4        -0.034249    -0.002361      0.169212   -0.324873  1.203297 -0.253485   
..             ...          ...           ...         ...       ...       ...   
321       0.165454     0.350670     -0.079472   -0.341807  0.670820  0.666460   
322      -0.480219    -0.572145     -0.079472    0.080503  1.788854  0.827053   
323      -0.480219    -0.387582     -0.368459   -0.271422 -0.447214 -0.377393   
324      -0.157383     0.166107     -0.223966    0.045310 -0.447214 -0.136504   
325      -0.480219    -0.572145     -0.368459   -0.482577 -0.447214 -0.859171   

          filename  label  

In [48]:
train, test = train_test_split(normal_df, test_size=0.2)
train_x = train[input_col].to_numpy()
train_y = tf.keras.utils.to_categorical(train['label'].to_numpy()-1)
test_x = test[input_col].to_numpy()
test_y = tf.keras.utils.to_categorical(test['label'].to_numpy()-1)

In [49]:
K.clear_session()
model = Sequential()
model.add(layers.Dense(60,activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Dense(30,activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Dense(30,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Dense(30,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(layers.Dense(15,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))          
#모델 컴파일
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=["accuracy"])

In [50]:
model.fit(train_x,train_y,epochs=100)
model.summary()
#모델 예측 및 평가
score = model.evaluate(test_x,test_y)
print("score:",score)


Epoch 1/100
9/9 [==============================] - 2s 2ms/step - loss: 1.1359 - accuracy: 0.1959
Epoch 2/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1271 - accuracy: 0.2362
Epoch 3/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1145 - accuracy: 0.3173
Epoch 4/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1107 - accuracy: 0.3100
Epoch 5/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1012 - accuracy: 0.3035
Epoch 6/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0964 - accuracy: 0.3186
Epoch 7/100
9/9 [==============================] - 0s 2ms/step - loss: 1.1092 - accuracy: 0.3066
Epoch 8/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0919 - accuracy: 0.3865
Epoch 9/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0870 - accuracy: 0.4190
Epoch 10/100
9/9 [==============================] - 0s 2ms/step - loss: 1.0933 - accuracy: 0.3768
Epoch 11/100
9/9 [===========

9/9 [==============================] - 0s 2ms/step - loss: 0.8923 - accuracy: 0.6209
Epoch 85/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8872 - accuracy: 0.6233
Epoch 86/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8940 - accuracy: 0.6030
Epoch 87/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9436 - accuracy: 0.5735
Epoch 88/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8344 - accuracy: 0.6610
Epoch 89/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8827 - accuracy: 0.6260
Epoch 90/100
9/9 [==============================] - 0s 2ms/step - loss: 0.8944 - accuracy: 0.6448
Epoch 91/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9128 - accuracy: 0.6035
Epoch 92/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9095 - accuracy: 0.6019
Epoch 93/100
9/9 [==============================] - 0s 2ms/step - loss: 0.9105 - accuracy: 0.6226
Epoch 94/100
9/9 [===============

In [42]:
score = model.evaluate(train_x,train_y)

9/9 [==============================] - 0s 1ms/step - loss: 0.8974 - accuracy: 0.9107
